In [ ]:
!pip install PyWavelets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 18.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
import pandas as pd
import pywt
import os
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import soundfile as sf
import wave
from scipy.io import wavfile
from scipy import signal
from scipy.signal import butter, filtfilt, find_peaks, lfilter, sosfiltfilt

In [ ]:
drive.mount('/content/drive') # Change if needed
database_path = "" # Write the raw_database/ path
denoized_augmented_database_path = "" # Write the denoized_database/ path
databases_infos_path = "" # Write a directory path where you are reading and saving .xlsx files with databases informations.

Mounted at /content/drive


In [ ]:
database_info = pd.read_excel(os.path.join(databases_infos_path, "database_info.xlsx")) # Carrega informações do dataset limpo

In [ ]:
print(database_info)

    Unnamed: 0 Subject ID                             BPM  suposed_duration  \
0            0         f1                 132-148-150-150                40   
1            1         f2         150-147-152-150-148-150                60   
2            2         f3              []-142-130-135-136                50   
3            3         f4  []-152-144-142-143-138-135-131                80   
4            4         f5         150-156-156-154-158-155                60   
..         ...        ...                             ...               ...   
95          99        f97         145-140-145-144-150-145                60   
96         100        f98         138-132-131-129-132-133                60   
97         101        f99         136-135-131-138-142-145                60   
98         102       f100     126-127-131-125-130-144-125                70   
99         103       f101                 150-152-151-147                40   

    num_channels  sample_width  frame_rate  num_fra

In [ ]:
# Check if audio is normalized
def verify_normalization(subject_id):
  path_audio_completo = os.path.join(database_path, subject_id + ".wav")
  audio, sr = librosa.load(path_audio_completo, sr=16000)

  max_amplitude = np.max(np.abs(audio))

  if max_amplitude <= 1.0:
      return True
  else:
      return False

lista_nao_normalizados = []
for index, coluna in database_info.iterrows():
    if not verify_normalization(coluna['Subject ID']):
      lista_nao_normalizados.append(coluna['Subject ID'])

print(lista_nao_normalizados)


[]


### Filters

In [ ]:
def normalize_audio(data):
    max_amplitude = np.max(np.abs(data))
    normalized_data = data / max_amplitude  # Normalize between [-1, 1]
    return normalized_data

In [ ]:
def rescale_audio(data, mult=100):
    return data * mult  # Rescale between [-100, 100]

In [ ]:
def butter_bandpass_filter(data, lowcut=20, highcut=120, order=6, fs=16000): # filtra a frequencia do audio na 6 ordem
    nyquist = 0.5 * fs # Nyquist ensures that the filter operates correctly within the limits of audible frequencies and prevents distortions during audio processing.
    low = lowcut / nyquist
    high = highcut / nyquist

    if not (0 < low < high < 1):
      raise ValueError("Invalid cutoff frequencies. Ensure that 0 < low < high < 1.")

    if not np.isfinite(data).all():
        raise ValueError("The input contains NaN or Inf values.")

    sos = butter(order, [low, high], btype='band', output='sos')
    y = sosfiltfilt(sos, data)

    if not np.isfinite(y).all():
        raise ValueError("The filter generated non-finite values (NaN or Inf) in the signal.")


    return y

In [ ]:
def estimate_sigma(detail_coeffs):
    return np.median(np.abs(detail_coeffs)) / 0.6745

def wavelet_filter(data, wavelet='coif4', level=7):
    coeffs = pywt.wavedec(data, wavelet, level=level) # wavelet decomposition
    sigma = estimate_sigma(coeffs[-1]) # Estimate the noise sigma using the detail coefficients at the first level
    threshold = sigma * np.sqrt(2 * np.log(len(data))) # Calculate the universal threshold
    thresholded_coeffs = coeffs.copy() # Apply soft thresholding to the detail coefficients
    for i in range(1, len(coeffs)):
        thresholded_coeffs[i] = pywt.threshold(coeffs[i], threshold, mode='soft')

    denoised_data = pywt.waverec(thresholded_coeffs, wavelet) # Reconstruct the signal using the thresholded coefficients


    denoised_data = denoised_data[:len(data)] # Truncate or pad the reconstructed signal to match the original length

    return denoised_data


In [ ]:
def apply_tranformations(file_path, transformations):
    data, sample_rate = librosa.load(file_path, sr=16000)

    transformed_data = data
    for transform in transformations:
        transformed_data = transform(transformed_data)

    return transformed_data

Assembling 10 second instances and placing them in a directory.

In [ ]:
# Delete the created files that are in the output folder.

# def apagar_conteudo_diretorio(diretorio):
#     # Verifica se o diretório existe
#     if os.path.exists(diretorio):
#         # Lista todos os arquivos e subdiretórios no diretório
#         for filename in os.listdir(diretorio):
#             file_path = os.path.join(diretorio, filename)
#             try:
#                 # Se for um arquivo ou link simbólico, apaga
#                 if os.path.isfile(file_path) or os.path.islink(file_path):
#                     os.unlink(file_path)
#                 # Se for um diretório, apaga recursivamente
#                 elif os.path.isdir(file_path):
#                     apagar_conteudo_diretorio(file_path)
#                     os.rmdir(file_path)  # Remove o diretório vazio
#             except Exception as e:
#                 print(f"Erro ao apagar {file_path}. Detalhes: {e}")
#     else:
#         print(f"O diretório {diretorio} não existe.")

#apagar_conteudo_diretorio(denoized_augmented_database_path)

In [ ]:
def separate_audio(subject_id, bpm_complete):
    sr = 16000

    # Convert the BPM string into a list
    bpm_list = bpm_complete.split('-')

    # Read the full audio file and apply filters
    path_full_audio = os.path.join(database_path, subject_id + ".wav")
    audio = apply_tranformations(
        path_full_audio,
        [normalize_audio, butter_bandpass_filter, wavelet_filter]
    )

    # Duration of each segment in samples (e.g., 10 seconds)
    duration = 10 * sr

    # Temporarily store all segments (audio + bpm)
    segments = []
    start_audio_pointer = 0

    # Iterate through each BPM, extract the corresponding segment, and store it in 'segments'
    for bpm in bpm_list:
        if bpm != "[]":
            end_audio_pointer = start_audio_pointer + duration
            segment = audio[start_audio_pointer:end_audio_pointer]
            # Store tuple (segment, bpm_float)
            segments.append((segment, float(bpm)))
        else:
            # Mark as None if empty (we won't generate anything later)
            segments.append((None, None))
        start_audio_pointer += duration

    # Now we will save the original segments
    ids = []
    bpms = []
    segment_counter = 0

    for i, (segment, bpm) in enumerate(segments):
        if segment is not None:
            # Save the full segment
            output_path = os.path.join(
                denoized_augmented_database_path, f"{subject_id}_{segment_counter}_{bpm}.wav"
            )
            sf.write(output_path, segment, sr)

            ids.append(f"{subject_id}_{segment_counter}")
            bpms.append(bpm)
            segment_counter += 1

    # Now we create new segments: half of i and half of i+1
    # Only if both are not None
    for i in range(len(segments) - 1):
        seg1, bpm1 = segments[i]
        seg2, bpm2 = segments[i + 1]

        if seg1 is not None and seg2 is not None:
            # Take half of each segment
            half1 = seg1[:duration // 2]
            half2 = seg2[:duration // 2]
            # Merge both halves
            new_segment = np.concatenate([half1, half2])

            # Assign the average BPM (can be rounded if desired)
            new_bpm = (bpm1 + bpm2) / 2

            # Save this new segment
            output_path_mix = os.path.join(
                denoized_augmented_database_path, f"{subject_id}_{segment_counter}_{new_bpm}.wav"
            )
            sf.write(output_path_mix, new_segment, sr)

            # Update ID and BPM lists
            ids.append(f"{subject_id}_{segment_counter}")
            bpms.append(new_bpm)
            segment_counter += 1

    # Return a final DataFrame with all IDs and BPMs
    return pd.DataFrame({'ID': ids, 'BPM': bpms})


In [ ]:
lista_df_segmentos = []

for index, coluna in database_info.iterrows():
    df_segmentos = separate_audio(coluna['Subject ID'], coluna['BPM'])
    lista_df_segmentos.append(df_segmentos)

df = pd.concat(lista_df_segmentos, ignore_index=True)

In [ ]:
print(df.to_string())

            ID    BPM
0         f1_0  132.0
1         f1_1  148.0
2         f1_2  150.0
3         f1_3  150.0
4         f1_4  140.0
5         f1_5  149.0
6         f1_6  150.0
7         f2_0  150.0
8         f2_1  147.0
9         f2_2  152.0
10        f2_3  150.0
11        f2_4  148.0
12        f2_5  150.0
13        f2_6  148.5
14        f2_7  149.5
15        f2_8  151.0
16        f2_9  149.0
17       f2_10  149.0
18        f3_0  142.0
19        f3_1  130.0
20        f3_2  135.0
21        f3_3  136.0
22        f3_4  136.0
23        f3_5  132.5
24        f3_6  135.5
25        f4_0  152.0
26        f4_1  144.0
27        f4_2  142.0
28        f4_3  143.0
29        f4_4  138.0
30        f4_5  135.0
31        f4_6  131.0
32        f4_7  148.0
33        f4_8  143.0
34        f4_9  142.5
35       f4_10  140.5
36       f4_11  136.5
37       f4_12  133.0
38        f5_0  150.0
39        f5_1  156.0
40        f5_2  156.0
41        f5_3  154.0
42        f5_4  158.0
43        f5_5  155.0
44        

In [ ]:
def get_wav_info(file_name):
    with wave.open(file_name, 'rb') as wav_file:

        num_channels = wav_file.getnchannels()
        sample_width = wav_file.getsampwidth()
        frame_rate = wav_file.getframerate()
        num_frames = wav_file.getnframes()
        duration = num_frames / frame_rate
        return num_channels, sample_width, frame_rate, num_frames, duration

In [ ]:
num_channels_list = []
sample_width_list = []
frame_rate_list = []
num_frames_list = []
duration_list = []

for index, coluna in df.iterrows():
    file_name = os.path.join(denoized_augmented_database_path, f"{coluna['ID'] + '_' + str(coluna['BPM'])}.wav")
    num_channels, sample_width, frame_rate, num_frames, duration = get_wav_info(file_name)
    num_channels_list.append(num_channels)
    sample_width_list.append(sample_width)
    frame_rate_list.append(frame_rate)
    num_frames_list.append(num_frames)
    duration_list.append(duration)

df['num_channels'] = num_channels_list
df['sample_width'] = sample_width_list
df['frame_rate'] = frame_rate_list
df['num_frames'] = num_frames_list
df['duration'] = duration_list

In [ ]:
print(len(df))

1145


In [ ]:
print(df.to_string())

            ID    BPM  num_channels  sample_width  frame_rate  num_frames  duration
0         f1_0  132.0             1             2       16000      160000    10.000
1         f1_1  148.0             1             2       16000      160000    10.000
2         f1_2  150.0             1             2       16000      160000    10.000
3         f1_3  150.0             1             2       16000      128160     8.010
4         f1_4  140.0             1             2       16000      160000    10.000
5         f1_5  149.0             1             2       16000      160000    10.000
6         f1_6  150.0             1             2       16000      160000    10.000
7         f2_0  150.0             1             2       16000      160000    10.000
8         f2_1  147.0             1             2       16000      160000    10.000
9         f2_2  152.0             1             2       16000      160000    10.000
10        f2_3  150.0             1             2       16000      160000   

In [ ]:
print(df["num_channels"].value_counts())
print(df["sample_width"].value_counts())
print(df["frame_rate"].value_counts())
print(df["num_frames"].value_counts())
print(df["duration"].value_counts())

num_channels
1    1145
Name: count, dtype: int64
sample_width
2    1145
Name: count, dtype: int64
frame_rate
16000    1145
Name: count, dtype: int64
num_frames
160000    1134
138400       2
128160       1
134480       1
114720       1
14240        1
94240        1
132160       1
122880       1
153920       1
117120       1
Name: count, dtype: int64
duration
10.000    1134
8.650        2
8.010        1
8.405        1
7.170        1
0.890        1
5.890        1
8.260        1
7.680        1
9.620        1
7.320        1
Name: count, dtype: int64


In [ ]:
# Delete segments shorter than 10 seconds
instancias_com_pouca_duracao = df[df["duration"] < 10]

for index, coluna in instancias_com_pouca_duracao.iterrows():
  os.remove(os.path.join(denoized_augmented_database_path, f"{coluna['ID'] + '_' + str(coluna['BPM'])}.wav"))
  df = df.drop(index)


In [ ]:
print(df["num_channels"].value_counts())
print(df["sample_width"].value_counts())
print(df["frame_rate"].value_counts())
print(df["num_frames"].value_counts())
print(df["duration"].value_counts())

num_channels
1    1134
Name: count, dtype: int64
sample_width
2    1134
Name: count, dtype: int64
frame_rate
16000    1134
Name: count, dtype: int64
num_frames
160000    1134
Name: count, dtype: int64
duration
10.0    1134
Name: count, dtype: int64


In [ ]:
df.to_excel(os.path.join(databases_infos_path, "instances_augmented.xlsx"))